In [ ]:
# Mount to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install rtree
!pip install geopandas
!pip install osmnx
!pip install pygeos
!pip install matplotlib==3.1.3 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
osmnx 1.1.2 requires matplotlib>=3.4, but you have matplotlib 3.1.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import osmnx as ox
import pytz

from pygeos import predicates
from scipy.spatial import distance

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


# Calculations

In [ ]:
#Mean Calculations
import os
directory= ('/content/drive/MyDrive/Colab Notebooks/Data/shared_data_LUR/data/purpleair/41860/2021/')

month_strs=[]
pm_per_hour=[]
station_id_all_stations=[]
col_pm=[]
mean_of_station=[]
all_values_by_month=[]
all_stations_pm=[]


# Loop through months
for month in range(13):
  month_str = str(month)
  if month < 10:
    month_str = '0' + str(month)
  month_strs.append(month_str)
month_strs.remove("00")

for month in month_strs:
  directory_= directory + month + "/"
  for filename in os.listdir(directory_):
    # Read in data and loop through
    f = os.path.join(directory_ + filename)
    data = pd.read_csv(f)

    # Extract station id
    station_id= list(data.columns)[-1]
    ch = '_'
    station_id= station_id.split(ch, 1)
    station_id=station_id[1]
    if len(station_id) > 0: 
      strValue = station_id[1]
      station_id_all_stations.append(strValue)
    
    col_pm = data[data.columns[-1]] #Take the PM value (one hour of one month at one station)
    col_pm = col_pm.dropna(axis='rows')
    if col_pm.shape[0] > 0:
      mean_of_station=np.nanmean(col_pm)# Take average of each station 
      all_stations_pm.append([station_id, month, mean_of_station]) #Add the average to a dateframe for all stations for that month



In [ ]:
all_stations = pd.DataFrame (all_stations_pm, columns = ['Station ID', 'Month', "Average PM Value"])
print (all_stations)

      Station ID Month  Average PM Value
0          90821    01         10.627728
1          69563    01         17.975011
2          81275    01         10.861677
3          63605    01         16.043485
4          84247    01         13.009011
...          ...   ...               ...
20351      83179    12         13.302127
20352      62489    12         15.141570
20353      87337    12         15.511297
20354      19066    12         14.809015
20355      65251    12          1.901266

[20356 rows x 3 columns]


In [ ]:
all_stations=all_stations.groupby(['Station ID'])['Average PM Value'].mean().reset_index()

In [ ]:
df = pd.DataFrame(all_stations)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Averages/Averages.csv', index=False)
df

,Station ID,Average PM Value
0,100023,11.418096
1,100255,7.467523
2,10092,7.168577
3,100951,8.481201
4,101031,6.635202
...,...,...
1841,97901,8.279408
1842,97993,0.725725
1843,98065,5.805587
1844,98073,7.651976


# Add lat and long

In [ ]:
all_stations = pd.read_csv ("/content/drive/MyDrive/Colab Notebooks/Averages/Averages.csv")

In [ ]:
all_stations["Station ID"]

0       100023
1       100255
2        10092
3       100951
4       101031
         ...  
1841     97901
1842     97993
1843     98065
1844     98073
1845     98095
Name: Station ID, Length: 1846, dtype: int64

In [ ]:
# Match station id
latitude_all_stations=[]
longitude_all_stations=[]

stations= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/sensors_ca.csv", sep=",", encoding='cp1252')
san_fran_metro_sites = stations[stations['airDistrict'].str.contains("San Francisco Bay")] 
san_fran_metro_outside_sites = san_fran_metro_sites[san_fran_metro_sites['DEVICE_LOCATIONTYPE'].str.contains("outside", na=False)] 

list_of_stations= san_fran_metro_outside_sites[["ID", "latitude", "longitude"]]
list_of_stations["ID"]=list_of_stations["ID"].astype(str)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
list_of_stations['ID']=list_of_stations['ID'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
all_stations=all_stations.merge(list_of_stations, left_on='Station ID', right_on='ID')
all_stations

,Station ID,Average PM Value,ID,latitude,longitude
0,100023,11.418096,100023,37.655017,-122.473087
1,100255,7.467523,100255,37.898709,-122.701600
2,10092,7.168577,10092,37.857566,-121.972860
3,100951,8.481201,100951,37.436701,-122.202417
4,101031,6.635202,101031,37.760164,-122.470343
...,...,...,...,...,...
1803,97767,8.432782,97767,37.773954,-122.494069
1804,97883,7.990777,97883,37.909299,-122.516275
1805,97901,8.279408,97901,37.757695,-122.451075
1806,98073,7.651976,98073,37.775839,-122.488848


In [ ]:
all_stations = all_stations[['Station ID', 'latitude', 'longitude', 'Average PM Value']]
all_stations

,Station ID,latitude,longitude,Average PM Value
0,100023,37.655017,-122.473087,11.418096
1,100255,37.898709,-122.701600,7.467523
2,10092,37.857566,-121.972860,7.168577
3,100951,37.436701,-122.202417,8.481201
4,101031,37.760164,-122.470343,6.635202
...,...,...,...,...
1803,97767,37.773954,-122.494069,8.432782
1804,97883,37.909299,-122.516275,7.990777
1805,97901,37.757695,-122.451075,8.279408
1806,98073,37.775839,-122.488848,7.651976


In [ ]:
df = pd.DataFrame(all_stations)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Averages/Averages with Lat and Long.csv', index=False)
df

,Station ID,latitude,longitude,Average PM Value
0,100023,37.655017,-122.473087,11.418096
1,100255,37.898709,-122.701600,7.467523
2,10092,37.857566,-121.972860,7.168577
3,100951,37.436701,-122.202417,8.481201
4,101031,37.760164,-122.470343,6.635202
...,...,...,...,...
1803,97767,37.773954,-122.494069,8.432782
1804,97883,37.909299,-122.516275,7.990777
1805,97901,37.757695,-122.451075,8.279408
1806,98073,37.775839,-122.488848,7.651976
